In [1]:
#Imports the libraries and read the data files
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os, sys, email
import gensim
from gensim.models import Doc2Vec
from sklearn.cluster import AffinityPropagation
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pprint
import nltk
from string import punctuation
import timeit
from sklearn.cluster import KMeans
from sklearn import metrics
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
start = timeit.default_timer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kaushlesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
repo_df = pd.read_csv('./news_tagged.csv')

In [3]:
repo_df.head()

,News_Heading,Risk Tag
0,IMF chief Lagarde charged over French corrupt...,F
1,$1 Billion Agreement with SunTrust to Address ...,L
2,$1 million for GM victims' families,L
3,$1 million IRS phone scam 'largest ever',F
4,$1.2 B settlement with Toyota because of safet...,L


In [4]:
sentences_df = pd.read_csv('./news_tagged.csv')

In [5]:
sentences_df.head()
print(sentences_df[:10])

                                        News_Heading Risk Tag
0   IMF chief Lagarde charged over French corrupt...        F
1  $1 Billion Agreement with SunTrust to Address ...        L
2                $1 million for GM victims' families        L
3           $1 million IRS phone scam 'largest ever'        F
4  $1.2 B settlement with Toyota because of safet...        L
5  $1.2 Billion Settlement For Toyota's 2009-2010...        L
6  $1.2 billion Toyota penalty sends warning to a...        L
7  $1.27 Billion in Damages to be Paid by the Ban...        L
8  $1.2B fine to settle probe delivers relief for...        L
9                    $1.2B Toyota fine: GM's future?        L


In [6]:
sentences_df['Risk Tag'].value_counts()

O    5154
I    2190
E    1376
F    1249
K     793
G     528
L     489
N     480
P     450
S      84
C      76
T      54
H      37
A      25
M      17
Q       9
Name: Risk Tag, dtype: int64

In [7]:
sentences_df.iloc[0]

News_Heading     IMF chief Lagarde charged over French corrupt...
Risk Tag                                                        F
Name: 0, dtype: object

In [8]:
#Data cleaning
def sentence_cleaning(text):
    sentence = text.lower()    
    # clean and tokenize document string
    sentence_content = sentence.split()    
    word_list = []
    for i in sentence_content:
        x = 0
        if (('http' not in i) and ('@' not in i) and ('<.*?>' not in i) and i.isalnum() and (not i in stop_words)):
            word_list += [i]
        
    return word_list

#Data Pre-processing
def preprocessing(text):    
    # remove numbers
    number_tokens = [re.sub(r'[\d]', ' ', i) for i in text]
    number_tokens = ' '.join(number_tokens).split()
     # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in number_tokens]
    # remove empty
    length_tokens = [i for i in stemmed_tokens if len(i) > 1]
    return length_tokens

In [9]:
LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
all_content = []
#texts = []
j=0
k=0
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
p_stemmer = PorterStemmer()
for em in sentences_df.News_Heading:           
    #Data cleaning
    clean_content = sentence_cleaning(em)
    
    #Pre-processing
    processed_sentences = preprocessing(clean_content)
    #processed_sentences = clean_content
    
    # add tokens to list
    if processed_sentences:
        all_content.append(LabeledSentence1(processed_sentences,[k]))
        j+=1
        
    k+=1

print("Number of sentences processed: ", k)
print("Number of non-empty sentence vectors: ", j)

Number of sentences processed:  13011
Number of non-empty sentence vectors:  13006


In [10]:
print(all_content[0])
type(all_content)

TaggedDocument(['imf', 'chief', 'lagard', 'charg', 'french', 'corrupt', 'case'], [0])


list

In [11]:
d2v_model = Doc2Vec(vector_size = 50, window = 2, min_count = 1, workers=4, dm = 0, dbow_words = 1, negative = 15, alpha=0.025, min_alpha=0.0125)

In [12]:
d2v_model.build_vocab(all_content)

In [13]:
    from random import shuffle
    def randomize(sentence):
        x = [sentence[i] for i in range(len(sentence))]
        shuffle(x)
        return x
    

In [14]:
for epoch in range(10):
    d2v_model.train(randomize(all_content), total_examples=d2v_model.corpus_count, epochs=epoch, start_alpha=0.002, end_alpha=-0.016)

In [15]:
d2v_model.most_similar('sold')

/home/kaushlesh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hit', 0.9999307990074158),
 ('chines', 0.9999301433563232),
 ('data', 0.9999286532402039),
 ('lead', 0.9999277591705322),
 ('dog', 0.9999269247055054),
 ('add', 0.9999266862869263),
 ('investig', 0.9999265670776367),
 ('food', 0.9999265074729919),
 ('comput', 0.9999237060546875),
 ('gm', 0.9999231100082397)]

In [28]:
tokens = preprocessing(sentence_cleaning('Chinese Premier zero tolerance for corruption'))
print(tokens)
new_vector = d2v_model.infer_vector(tokens)
print(new_vector)
similar_doc = d2v_model.docvecs.most_similar(positive=[new_vector])
similar_doc = np.array(similar_doc)
print(similar_doc[:,0])
for l in similar_doc[:,0]:
    print(int(l))
    print(all_content[int(l)])
#print(all_content[558])

['chines', 'premier', 'zero', 'toler', 'corrupt']
[-0.6340072   0.7821126   0.02652645  0.617036   -0.51429796 -0.59344846
 -0.3886683  -0.05824026  0.97966164  1.2516298  -0.7472035  -1.2175776
 -0.2687559  -0.82236177  0.6248389   0.2911443  -0.19019815 -0.31820032
 -0.8150261  -0.19659632  0.4825385   0.8837144  -0.64621097  0.34890807
 -0.01100883 -0.3017885  -1.5205872  -0.1889602  -0.27852216 -0.54767174
  2.0140169  -0.01873509  0.2634091   0.05573512  1.0091954  -0.51070756
 -0.33269212 -0.19227661  0.43911564  0.10876785  0.13284042  0.10723174
  0.59501755  0.65203077  0.08171245 -0.24769133 -0.6480989  -0.05637979
  0.8334703   0.46032333]
[12188.   558. 10849.  6571.   617. 10425.  5592. 11644. 12540.  7149.]
12188
TaggedDocument(['ico', 'probe', 'facebook', 'controversi', 'user', 'experi'], [12193])
558
TaggedDocument(['view', 'crowdfund', 'environment'], [559])
10849
TaggedDocument(['say', 'halifax', 'protest'], [10853])
6571
TaggedDocument(['nest', 'recal', 'smoke', 'ala

In [29]:
#saving the created model
d2v_model.save('d2v_model')
print('model saved')

model saved


In [30]:
import gc
gc.collect()

15

In [31]:
d2v_model.delete_temporary_training_data()

In [32]:
model_loaded = Doc2Vec.load('d2v_model')

In [33]:
clustering = AffinityPropagation()

In [34]:
clustering

AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)

In [35]:
clustering.fit(model_loaded.docvecs.vectors_docs)

AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)

In [36]:
labels=clustering.labels_.tolist()
type(labels)

list

In [37]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(d2v_model.docvecs.doctag_syn0, labels, metric='sqeuclidean'))

/home/kaushlesh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  


Silhouette Coefficient: 0.013
